In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

Using TensorFlow backend.


In [2]:
index_df = pd.read_csv('../data/xeno-canto_ca-nv_stats.csv', 
                       index_col='file_id')
index_df.head(1)

,index,country,duration_seconds,english_cname,file_name,file_url,genus,latitude,license,location,...,meldb_mean123,meldb_std123,meldb_mean124,meldb_std124,meldb_mean125,meldb_std125,meldb_mean126,meldb_std126,meldb_mean127,meldb_std127
file_id,,,,,,,,,,,,,,,,,,,,,
17804,0,United States,3,Abert's Towhee,XC17804.mp3,https://www.xeno-canto.org/17804/download,Melozone,33.3117,http://creativecommons.org/licenses/by-nc-nd/2.5/,"Cibola National Wildlife Refuge, Cibola, Arizo...",...,-79.65696,0.863908,-79.742508,1.001422,-79.764236,0.831335,-78.878372,1.854941,-79.423218,1.264777


In [3]:
train_df = index_df.loc[pd.read_csv('../data/train_file_ids.csv')['file_id']]
len(train_df)

1820

In [4]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['english_cname'])
#y_train = to_categorical(y_train)
label_encoder.classes_, y_train

(array(["Abert's Towhee", 'Acorn Woodpecker', 'American Bushtit',
        'American Crow', 'American Dusky Flycatcher',
        'American Grey Flycatcher', 'American Robin', "Anna's Hummingbird",
        'Ash-throated Flycatcher', "Bell's Sparrow", "Bell's Vireo",
        "Bewick's Wren", 'Black Phoebe', 'Black-chinned Sparrow',
        'Black-headed Grosbeak', 'Black-tailed Gnatcatcher',
        'Blue-grey Gnatcatcher', "Brewer's Sparrow",
        'California Gnatcatcher', 'California Quail',
        'California Scrub Jay', 'California Thrasher', 'California Towhee',
        'Canyon Wren', "Cassin's Finch", "Cassin's Vireo",
        'Chestnut-backed Chickadee', "Clark's Nutcracker",
        'Common Poorwill', 'Common Yellowthroat', 'Dark-eyed Junco',
        'Elegant Tern', 'Flammulated Owl', "Forster's Tern",
        'Great Horned Owl', 'Green-tailed Towhee', 'Grey Vireo',
        'Hermit Thrush', 'Hermit Warbler', 'House Finch', 'House Wren',
        "Hutton's Vireo", 'Juniper Titmo

In [5]:
X_train = train_df[#['meldb_mean%03i'%i for i in range(128)]+\
                 #['meldb_std%03i'%i for i in range(128)]+\
                 ['mfcc_mean%02i'%i for i in range(20)]+\
                 ['mfcc_std%02i'%i for i in range(20)]]
X_train

,mfcc_mean00,mfcc_mean01,mfcc_mean02,mfcc_mean03,mfcc_mean04,mfcc_mean05,mfcc_mean06,mfcc_mean07,mfcc_mean08,mfcc_mean09,...,mfcc_std10,mfcc_std11,mfcc_std12,mfcc_std13,mfcc_std14,mfcc_std15,mfcc_std16,mfcc_std17,mfcc_std18,mfcc_std19
file_id,,,,,,,,,,,,,,,,,,,,,
453098,-250.810425,28.457371,9.712974,-1.897048,1.805519,-8.206538,-3.683885,-3.442959,4.030917,-1.564035,...,10.730343,11.178503,9.452717,7.915398,7.054220,6.918440,7.249106,7.347002,8.047519,7.091804
344803,-158.820038,58.932842,-75.206795,-2.338842,-11.465850,-4.717530,10.324313,0.330541,-5.667123,-2.570280,...,8.559773,8.694917,7.524657,7.227513,6.131079,6.439923,7.126657,6.647209,6.963773,6.617993
355425,-341.699921,57.504631,-15.516808,2.974963,-7.245710,-2.083582,2.025671,9.330463,-2.084751,-4.569627,...,8.579488,5.924883,7.375512,5.640487,4.680757,6.270146,5.649363,6.573978,7.053258,5.101687
451191,-370.928314,6.029840,4.018046,-4.656692,6.181147,-10.600257,4.310547,2.256558,-1.133137,-1.591350,...,11.808133,9.050931,7.650569,8.156438,6.276405,6.831277,7.031040,6.600980,7.742706,6.687588
219357,-375.992889,37.905609,-38.414371,-13.534421,22.002016,-5.593565,-5.510320,3.000481,3.125098,-3.365547,...,9.230787,8.730521,7.532840,6.774899,6.296411,5.977695,6.050071,6.386250,6.742312,5.907849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305603,-331.223053,112.820686,-57.858753,-6.268742,-19.864056,-2.324106,-7.908581,-2.829157,-4.857272,-1.182806,...,4.346152,3.940823,3.976696,4.286355,4.425521,3.699108,4.473426,4.427166,3.953110,3.538882
405020,-371.174957,3.633566,-52.221279,47.875305,-58.673878,-2.351454,12.076879,-7.640750,-1.003304,-2.550456,...,6.446206,5.673038,5.748996,6.285236,5.290792,5.231864,5.760043,5.130857,5.232782,5.399832
326580,-704.942200,38.930153,-10.325354,18.968468,9.318748,-7.683753,1.563334,6.615032,-0.264179,-2.896008,...,11.942576,9.026079,12.098244,7.742556,8.776327,7.777817,7.647707,7.062777,6.859518,6.143765


In [6]:
scaler = StandardScaler().fit(X_train)

In [7]:
X_train = scaler.transform(X_train)
X_train

array([[ 1.07232185, -0.55056792,  0.96675712, ...,  0.62615872,
         0.92080878,  0.65581414],
       [ 1.90967676,  0.05401906, -1.63103234, ...,  0.21160539,
         0.34708893,  0.36157679],
       [ 0.24498795,  0.02568552,  0.19495026, ...,  0.16822363,
         0.39446137, -0.5800512 ],
       ...,
       [-3.06147391, -0.34280386,  0.35376257, ...,  0.45778516,
         0.29189795,  0.06708092],
       [ 0.88090373,  2.17849718, -0.04244931, ...,  0.26814712,
        -0.14022308, -0.14706461],
       [-0.12985793, -1.14745891,  0.43240696, ..., -0.70030109,
        -0.73636208, -0.40819022]])

In [8]:
X_train.shape, y_train.shape

((1820, 40), (1820,))

## Train with Cross-Validation

### Train a Gaussian Naive Bayes Classifier

In [9]:
n_splits = 5
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1/3, random_state=37)
scores = []

for cv_train_index, cv_val_index in sss.split(X_train, y_train):
    predictor = GaussianNB().fit([X_train[i] for i in cv_train_index], 
                                     [y_train[i] for i in cv_train_index])
    
    X_val = [X_train[i] for i in cv_val_index] 
    y_val = [y_train[i] for i in cv_val_index]
    
    predictions = predictor.predict(X_val)
    
    scores.append(accuracy_score(y_val, predictions))
    print("Split %i: score=%.5f"%(
        len(scores), scores[-1] ))
print("GaussianNB Cross Validation Accuracy: %.1f%% +/- %.2f%%"%(
    np.mean(scores)*100, np.std(scores)*100))

Split 1: score=0.16969
Split 2: score=0.18946
Split 3: score=0.16969
Split 4: score=0.16310
Split 5: score=0.16145
GaussianNB Cross Validation Accuracy: 17.1% +/- 1.00%


### Train a Multi-Layer Perceptron (Neural Network)

In [10]:
n_splits = 5
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1/3, random_state=37)
scores = []

for cv_train_index, cv_val_index in sss.split(X_train, y_train):
    predictor = MLPClassifier(hidden_layer_sizes=(256,256), max_iter=500).fit(
        [X_train[i] for i in cv_train_index], [y_train[i] for i in cv_train_index])
    
    X_val = [X_train[i] for i in cv_val_index] 
    y_val = [y_train[i] for i in cv_val_index]
    
    predictions = predictor.predict(X_val)
    
    scores.append(accuracy_score(y_val, predictions))
    print("Split %i: score=%.5f"%(
        len(scores),scores[-1] ))
print("MLP Cross Validation Accuracy: %.1f%% +/- %.2f%%"%(
    np.mean(scores)*100, np.std(scores)*100))

Split 1: score=0.24053
Split 2: score=0.29489
Split 3: score=0.24053
Split 4: score=0.23888
Split 5: score=0.24547
MLP Cross Validation Accuracy: 25.2% +/- 2.15%


## Calculate Test Score

### Train on whole training dataset

In [13]:
gnb =  MLPClassifier(hidden_layer_sizes=(256,256), max_iter=500).fit(X_train, y_train)

### Make predictions and report accuracy

In [14]:
test_df = index_df.loc[pd.read_csv('../data/test_file_ids.csv')['file_id']]
len(test_df)

910

In [15]:
X_test = test_df[['mfcc_mean%02i'%i for i in range(20)]+\
                 ['mfcc_std%02i'%i for i in range(20)]]
X_test = scaler.transform(X_test)
X_test.shape

(910, 40)

In [16]:
y_test = label_encoder.transform(test_df['english_cname'])
y_test.shape, y_test[:10], '...'

((910,), array([47, 65, 83, 42, 52, 65, 18, 33, 62, 35]), '...')

In [17]:
predictions = gnb.predict(X_test)
test_accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy: %.2f %%"%(100*test_accuracy))

Test Accuracy: 28.79 %
